You can access the accompanying video here: 

<a href="https://blinks.bloomberg.com/screens/PLYR%20VOD%20361824358"> Equity Fundamental Data and Box Plots (11 min)</a>

In [6]:
# First we want to import the BQL Library
import bql 
bq = bql.Service()

In [7]:
# This whole section is creating a dataframe based on BQL for our quartile plot

# Define our fundamental metric
metric = (bq.data.bs_cash_near_cash_item()/bq.data.sales_rev_turn())*100

# Define which country IDs fit into which region bucket
northam = bq.func.value(bq.data.ID(),bq.univ.members('NorthAmerica',type='Region'))
asia = bq.func.value(bq.data.ID(),bq.univ.members('Asia',type='Region'))
easteur = bq.func.value(bq.data.ID(),bq.univ.members('EasternEurope',type='Region'))
latam = bq.func.value(bq.data.ID(),bq.univ.members('SouthCentralAmerica',type='Region'))
australiaocea = bq.func.value(bq.data.ID(),bq.univ.members('AustraliaOceania',type='Region'))
africa = bq.func.value(bq.data.ID(),bq.univ.members('SubsaharanAfrica',type='Region'))
westeu = bq.func.value(bq.data.ID(),bq.univ.members('WesternEurope',type='Region'))
mena = bq.func.value(bq.data.ID(),bq.univ.members('MENA',type='Region'))

# Define our regions grouping through a nested IF statement 
regions_grouping = bq.func.if_(bq.func.any(bq.data.cntry_of_domicile()==northam),'NorthAmerica',
                              bq.func.if_(bq.func.any(bq.data.cntry_of_domicile()==asia),'Asia',
                            bq.func.if_(bq.func.any(bq.data.cntry_of_domicile()==easteur),'EasternEurope',
                            bq.func.if_(bq.func.any(bq.data.cntry_of_domicile()==latam),'Latam',
                            bq.func.if_(bq.func.any(bq.data.cntry_of_domicile()==australiaocea),'australiaocea',
                            bq.func.if_(bq.func.any(bq.data.cntry_of_domicile()==africa),'africa',
                            bq.func.if_(bq.func.any(bq.data.cntry_of_domicile()==westeu),'westerneurope',
                            bq.func.if_(bq.func.any(bq.data.cntry_of_domicile()==mena),'MENA',
                                       'Other'))))))))

# Specify our industry 
industry = 'Airlines'

# Create our quartile values for each group 
lower = bq.func.quantile(bq.func.group(metric['VALUE'],regions_grouping),0.05)['VALUE']
first_quartile = bq.func.quantile(bq.func.group(metric['VALUE'],regions_grouping),0.25)['VALUE']
median = bq.func.median(bq.func.group(metric['VALUE'],regions_grouping))['VALUE']
third_quartile = bq.func.quantile(bq.func.group(metric['VALUE'],regions_grouping),0.75)['VALUE']
upper = bq.func.quantile(bq.func.group(metric['VALUE'],regions_grouping),0.95)['VALUE']

# Define which items we want to pull via BQL with a dictionary
items = {'Lower':lower,
        '25% Quartile':first_quartile,
        'Median':median,
        '75% Quartile':third_quartile,
        'Upper':upper}

# Create our filtered universe 
univ = bq.univ.filter(bq.univ.equitiesuniv(['Active','Primary']),
                     bq.func.and_(bq.data.classification_name('BICS','3')==industry,bq.data.cur_mkt_cap()>'10M'))

# Create our Global parameters
with_clause = {'Mode':'Cached','Currency':'USD','fill':'prev'}

# Create, execute, and put our request into a dataframe
req = bql.Request(univ,items,with_clause)
resp = bq.execute(req)
df = bql.combined_df(resp)

In [8]:
df

,Lower,25% Quartile,Median,75% Quartile,Upper
ID,,,,,
Asia,1.188388,5.897896,11.697369,21.924178,30.865876
EasternEurope,2.068726,2.421516,4.932701,15.652713,32.109274
Latam,5.953571,6.316344,7.410415,8.445677,12.978971
MENA,9.970356,11.852471,13.757159,25.846877,50.397668
NorthAmerica,1.531759,6.595903,12.495464,16.917262,25.049157
africa,0.073383,0.232804,0.432080,3.184667,5.386737
australiaocea,3.318876,4.201203,9.673223,17.476229,18.734576
westerneurope,2.661309,7.124290,15.975645,32.126479,51.433146


In [9]:
# Create our outlier dataframe and define the output columns via a dictionary
condition = {'Value':bq.func.if_(bq.func.or_(metric['VALUE']<bq.func.ungroup(lower),
                                            metric['VALUE']>bq.func.ungroup(upper)),metric['VALUE'],'Nan').dropna(True),
            'Region':regions_grouping['VALUE']}

# Create, execute, and put our outlier request into a dataframe
outlier_request = bql.Request(univ,condition,with_clause)
outlier_resp = bq.execute(outlier_request)
outlierdf = bql.combined_df(outlier_resp).dropna().reset_index()

In [10]:
# Copy and Pasted code from the visualizations Gallery
import bqplot as bqp
import pandas as pd
import numpy as np

# Generate some data to work with
dataframe = df

outlier_df = outlierdf

# Get min/max values to provide to scales
min_val = min([dataframe['Lower'].min(), outlier_df['Value'].min()])
max_val = max([dataframe['Upper'].max(), outlier_df['Value'].max()])

# Create scales
scale_x = bqp.OrdinalScale()
scale_y = bqp.LinearScale(min=float(min_val),
                          max=float(max_val))

# Create Boxplot mark
mark_box = bqp.Boxplot(x=dataframe.index,
                       y=[dataframe.iloc[i] for i in range(len(dataframe))],
                       colors=['#1B84ED'],
                       scales={'x': scale_x, 'y': scale_y},
                       stroke='white',
                       auto_detect_outliers=False)

# Create Scatter to display outliers
mark_scatter = bqp.Scatter(x=outlier_df['Region'],
                           y=outlier_df['Value'],
                           colors=['red'],
                           # Assign data to an attribute to display in tooltip
                           size=outlier_df['ID'],
                           scales={'x': scale_x, 'y': scale_y},
                           tooltip=bqp.Tooltip(fields=['y', 'size'],
                                               formats=['.2f'],
                                               show_labels=False))

# Create Axes
axis_x = bqp.Axis(scale=scale_x)
axis_y = bqp.Axis(scale=scale_y,
                  tick_format='0.2f',
                  label='Value',
                  orientation='vertical')

# Create Figure
figure = bqp.Figure(marks=[mark_box, mark_scatter],
                    axes=[axis_x, axis_y],
                    layout={'width':'100%'},
                    title='Boxplot from precomputed statistics',
                    title_style={'font-size': '22px'},
                    fig_margin={'top': 60, 'bottom': 30,
                                'left': 70, 'right': 50})

# Display the figure
figure


Figure(axes=[Axis(scale=OrdinalScale()), Axis(label='Value', orientation='vertical', scale=LinearScale(max=60.…